# Cosmin Madalin Zaharia

### Implementation of findpeak and meanshift optimazed

In [1]:
import scipy.io
import os
import pandas as pd
import numpy as np
import cv2
import time
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist
from skimage.color import lab2rgb, rgb2lab
from sklearn.metrics.pairwise import euclidean_distances

## Run all the cells until to the test part

<i><b>mean_cluster_and_basin</b></i> is function called by find_peak_opt used to find the cluster where a pixel belong and all the poins to associate to the same cluster (basin) during the search.

In [2]:
def mean_cluster_and_basin(data, point, r, c):
        
         #calculate the distance between the dataset and the point
        distance = cdist(data.T, point.reshape((-1, 1)).T)
        
        #Now let's calculate the mean
        #Since I want to get a (3,) vector, the axis = 1
        centroid_point = np.mean(data[:, distance[:, 0] < r], axis=1)
        
        #calculate the basin
        distance_to_mean = cdist(data.T, centroid_point.reshape((-1, 1)).T)
        
        #selects only those columns whose distance from the mean point is less than r/c.
        basin = np.argwhere(distance_to_mean[:, 0] < r/c)[:,0]

        return centroid_point, basin

According to the homework: here have a the find_peak_opt function, the same as in the previus notebook but optimazed and with speed up

In [3]:
def find_peak_opt(data, idx, r, tau=0.01, c=4):
        
        
        point = data[:,idx]
        mean_point, basin =  mean_cluster_and_basin(data, point, r, c)
        
        #create a basin point to recover the points
        basin_points = set(basin)
        
        while pdist(np.array([point, mean_point])) > tau:
            point = mean_point
            mean_point, basin =  mean_cluster_and_basin(data, point, r, c)

            basin_points.update(basin)
        
        #this one is for taking those points <r
        _, basin =  mean_cluster_and_basin(data, point, r, 1)
        basin_points.update(basin)
        
        return np.array(mean_point), basin_points

Same meansfhit function but optimised

In [4]:
def meanshift_opt (data, r, c):
    labels = np.zeros(data.shape[1])-1 
    peaks = []
    
    peak, basin = find_peak_opt(data, 0 , r, 0.01, c)
    labels[0] = 0
    peaks.append(peak)
    
    for idx in tqdm(range(1, data.shape[1])):
        if labels[idx] == -1:
            peak, basin = find_peak_opt(data, idx, r, 0.01, c)
            
            #after you got the peak calculate the distance with the other peaks and see if it is smaller than r<2
            distance= cdist(np.array(peaks), np.array([peak]))
            peaks_around = np.argwhere(distance < r/2)

            if len(peaks_around) > 0:
                labels[idx] = peaks_around[0][0]
                labels[list(basin)] = peaks_around[0][0]
            else:
                 #this means that we aren't the window so the peak doesn't belong to that cluster => we found a new cluster
                labels[idx] = len(peaks)
                labels[list(basin)] = len(peaks)
                peaks.append(peak)

    return labels, peaks

<br>

Here we have the implementation of the segmIm(im, r): renamed since I've also added the pre_processing part in the same code

In [5]:
def pre_processing_and_segmentation(path, spatial_position, r, c):
    
    #read the image and covert in it into CIELAB color values
    image = plt.imread(path)
    image = rgb2lab(image)
    height, width, num_channels = image.shape
    
    
    if not spatial_position: 
        processed_image = image.reshape((height * width, num_channels)).T 
    
    if spatial_position:
        features = np.zeros((height,width, 5))
        for y in range(height):
            for x in range(width):
                #add the spatial information
                features[y, x] = np.append(image[y, x], [y, x]) 
        height, width, num_channels = features.shape
        processed_image = features.reshape((height*width,  num_channels)).T
    
    start_time = time.time()
    labels, peaks = meanshift_opt(processed_image, r, c)
    end_time = time.time()
    
    # get the execution time
    elapsed_time = end_time - start_time
    
    copy_peaks = peaks.copy()
    copy_labels = labels.copy()
    
    # convert copy_peaks to integers
    copy_peaks = np.array(copy_peaks).astype(int)

    # reshape the array
    matrix_reshaped = processed_image.T
    
    for i, pixel in enumerate(matrix_reshaped):
        matrix_reshaped[i] = copy_peaks[int(copy_labels[i])]


    # unfortnuatelly I don't know why cv2 doesn't work for with 5D images
    if spatial_position:
        segmented = lab2rgb(matrix_reshaped.reshape(features.shape)[:, :, :3])
    else:
        segmented = lab2rgb(matrix_reshaped.reshape(image.shape))

    
    plt.imshow(segmented)
    plt.suptitle("Execution Time: {:.2f} seconds and #_clusters =  {}".format(elapsed_time, str(len(peaks))), y = 1.02)

    if spatial_position:
        plt.title("Image 5D with c = {} and r = {}".format(c, r), y= 1.02)
        plt.savefig(os.path.join('image_plots', path + "5d_segmented_image_c{}_r{}.png".format(c, r)),  bbox_inches='tight')
    else:
        plt.title("Image 3D with c = {} and r = {}".format(c, r),  y= 1.02)
        plt.savefig(os.path.join('image_plots', path + "3d_segmented_image_c{}_r{}.png".format(c, r)),  bbox_inches='tight')
    
    plt.show() 


<br>

## Test

here you can test the code. Just few information:
<li>Punt the name of the file and be sure to be to have the image in the same directory of the notebook</li>
<li>False = 3D and True = 5D </li>
<li>The third parameter is r</li>
<li>The fourth parameter is c</li>

In [ ]:
pre_processing_and_segmentation('first_image.jpeg', False, 32, 4)

<br>

<br>

This code is here if you want to see the pixel distribution of your image

In [ ]:
img = plt.imread('london.jpeg')
img_arr = img
x, y, z = img.shape
img_points = np.zeros((x * y, 3))
for i in range(x):
    for j in range(y):
        img_points[i * y + j] = img_arr[i][j]

# Plot the 3D point cloud of the RGB image
fig = plt.figure(figsize=(8, 8))  # Increase figure size
fig.patch.set_facecolor('white')  # Set figure background color to white

ax = fig.add_subplot(111, projection='3d')
ax.scatter(img_points[:,0], img_points[:,1], img_points[:,2], c=img_points/255.0, s=1, alpha=0.5)

ax.set_facecolor('white')  # Set axes background color to white
ax.view_init(20, - 60)  # Rotate the plot

ax.set_xlabel('Red')
ax.set_ylabel('Green')
ax.set_zlabel('Blue')

plt.savefig(os.path.join('3d_plot_third_image'))

plt.show()